# 🖥️ Nested Grids

In some applications, you may have access to fields on different grids that each cover only part of the region of interest. Then, you would like to combine them all together. You may also have a grid covering the entire region and another one only covering part of it, but with a higher resolution. The set of those grids form what we call nested grids.

In Parcels v4, we can use the new `uxarray` integration to determine in which grid a particle is located. We will demonstrate how to set up a simulation with multiple nested grids, and how to handle particle transitions between these grids.


This tutorial shows how to use these Nested Fields with a very idealised example.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import numpy as np
import uxarray as ux
import xarray as xr
from shapely.geometry import MultiPoint, Point, Polygon
from triangle import triangulate

import parcels

## Setting up the individual Grids and Fields

First define a zonal and meridional velocity field defined on a high resolution (dx = 100m) 2kmx2km grid with a flat mesh. The zonal velocity is uniform and 1 m/s, and the meridional velocity is equal to 0.5 * cos(lon / 200 * pi / 2) m/s.

In [ ]:
from parcels._datasets.structured.generated import simple_UV_dataset

xdim = 10
ydim = 10

ds_in = [None] * 2
grid_polygons = [None] * 2

grid_polygons[0] = Polygon([(0, 0), (2000, 0), (2000, 2000), (0, 2000)])
ds_in[0] = simple_UV_dataset(dims=(1, 1, ydim, xdim), mesh="flat").isel(time=0, depth=0)

ds_in[0]["lon"][:] = np.linspace(
    grid_polygons[0].bounds[0], grid_polygons[0].bounds[2], xdim
)
ds_in[0]["lat"][:] = np.linspace(
    grid_polygons[0].bounds[1], grid_polygons[0].bounds[3], ydim
)
lon_g, lat_g = np.meshgrid(ds_in[0]["lon"], ds_in[0]["lat"])
ds_in[0]["U"][:] = 1.0
ds_in[0]["V"][:] = np.cos(lon_g / 200 * np.pi / 2)

Now define another velocity field on a low resolution (dx = 2km) 20kmx4km grid.

In [ ]:
xdim = 40
ydim = 20

grid_polygons[1] = Polygon(
    [(-2000, -1000), (18000, -1000), (18000, 3000), (-2000, 3000)]
)
ds_in[1] = simple_UV_dataset(dims=(1, 1, ydim, xdim), mesh="flat").isel(time=0, depth=0)

ds_in[1]["lon"][:] = np.linspace(
    grid_polygons[1].bounds[0], grid_polygons[1].bounds[2], xdim
)
ds_in[1]["lat"][:] = lat = np.linspace(
    grid_polygons[1].bounds[1], grid_polygons[1].bounds[3], ydim
)
lon_g, lat_g = np.meshgrid(ds_in[1]["lon"], ds_in[1]["lat"])
ds_in[1]["U"][:] = 1.0
ds_in[1]["V"][:] = np.cos(lon_g / 200 * np.pi / 2)

Plot the two velocity fields on top of each other, indicating the grid boundaries in red.

In [ ]:
n_grids = len(ds_in)
fig, ax = plt.subplots()

for i in range(n_grids):
    ds = ds_in[i]
    ax.quiver(ds.lon, ds.lat, ds.U, ds.V)
    ax.plot(
        np.append(grid_polygons[i].exterior.xy[0], grid_polygons[i].exterior.xy[0][0]),
        np.append(grid_polygons[i].exterior.xy[1], grid_polygons[i].exterior.xy[1][0]),
        "-r",
        lw=2,
    )

plt.tight_layout()
plt.show()

## Creating a Delaunay triangulation of the nested Grids

Now comes the important part: we need to create a Delaunay triangulation of the  nested Grids, so that we can efficiently determine in which Grid a particle is located at any given time. We use the `triangle` package to perform the triangulation, and `shapely` to handle the geometric operations. 

Note that we need to keep the edges of the polygons in the triangulation, so we need a [constrained (PSLG) Delaunay triangulation](https://en.wikipedia.org/wiki/Constrained_Delaunay_triangulation).

The result is a set of triangles covering the nested Grids, which we can use to determine in which Grid a particle is located at any given time. It is important that the list of polygons is ordered from smallest to largest Grid, so that triangles in overlapping areas are assigned to the correct Grid.

In [ ]:
import numpy as np
from shapely.geometry import Point, Polygon
from triangle import triangulate


def constrained_triangulate_keep_edges(shapely_polys):
    """Constrained triangulation while keeping polygon edges.

    Accepts a list of shapely.geometry.Polygon objects.

    Returns:
      pts (P,2) array of vertex coordinates,
      tris (M,3) array of triangle vertex indices,
      face_poly (M,) mapping each triangle to the polygon index or -1.
    """
    # normalize input: convert shapely Polygons to Nx2 numpy arrays
    np_polys = []
    for p in shapely_polys:
        coords = np.asarray(p.exterior.coords)
        if coords.shape[0] > 1 and np.allclose(coords[0], coords[-1]):
            coords = coords[:-1]
        np_polys.append(coords.astype(float))

    # build vertices + segments for Triangle PSLG
    verts = []
    segments = []
    offset = 0
    for poly in np_polys:
        Ni = len(poly)
        verts.extend(poly.tolist())
        segments.extend([[offset + j, offset + ((j + 1) % Ni)] for j in range(Ni)])
        offset += Ni
    verts = np.asarray(verts, dtype=float)
    segments = np.asarray(segments, dtype=int)

    mode = "p"  # "p" = PSLG (constrained triangulation)
    B = triangulate({"vertices": verts, "segments": segments}, mode)

    pts = B["vertices"]
    tris = B["triangles"].astype(int)

    # assign triangles to polygons using centroid test
    centers = pts[tris].mean(axis=1)
    face_poly = np.full(len(tris), -1, dtype=int)
    for ti, c in enumerate(centers):
        for ip in range(len(shapely_polys)):
            if shapely_polys[ip].contains(Point(c)):
                face_poly[ti] = ip
                break

    return pts, tris, face_poly

We can then run the triangulation and plot the resulting triangles to verify that they correctly cover the nested Grids.

In [ ]:
points, face_tris, face_poly = constrained_triangulate_keep_edges(grid_polygons)

fig, ax = plt.subplots()
for i in range(n_grids)[::-1]:
    tris = face_tris[face_poly == i]
    ax.triplot(points[:, 0], points[:, 1], tris, label=f"Nest {i}")
ax.scatter(points[:, 0], points[:, 1], s=10, c="k")
ax.set_aspect("equal")
ax.legend()

# reverse legend labels
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])
plt.show()

Then, we convert the triangulation into a Parcels FieldSet using `Parcels.UxGrid()`.

In [ ]:
# build an xarray dataset compatible with UGRID / uxarray
n_node = points.shape[0]
n_face = face_tris.shape[0]
n_max_face_nodes = face_tris.shape[1]

ds_tri = xr.Dataset(
    {
        "node_lon": (("n_node",), points[:, 0]),
        "node_lat": (("n_node",), points[:, 1]),
        "face_node_connectivity": (("n_face", "n_max_face_nodes"), face_tris),
        "face_polygon": (
            (
                "time",
                "nz",
                "n_face",
            ),
            face_poly[np.newaxis, np.newaxis, :],
            {
                "long_name": "Nested grid ID",
                "location": "face",
                "mesh": "delaunay",
            },
        ),
    },
    coords={
        "time": np.array([np.timedelta64(0, "ns")]),
        "nz": np.array([0]),
        "n_node": np.arange(n_node),
        "n_face": np.arange(n_face),
    },
    attrs={"Conventions": "UGRID-1.0"},
)

uxda = ux.UxDataArray(ds_tri["face_polygon"], uxgrid=ux.Grid(ds_tri))

NestID = parcels.Field(
    "NestID",
    uxda,
    parcels.UxGrid(uxda.uxgrid, z=uxda["nz"], mesh="flat"),
    interp_method=parcels.interpolators.UxPiecewiseConstantFace,
)
fieldset = parcels.FieldSet([NestID])

We can confirm that the FieldSet has been created correctly by running a Parcels simulation where particles sample the `NestID` field, which indicates in which nest each particle is located at any given time.

In [ ]:
X, Y = np.meshgrid(
    np.linspace(-1000, 5000, 10),
    np.linspace(-500, 2500, 10),
)

NestParticle = parcels.Particle.add_variable(parcels.Variable("nestID", dtype=np.int32))
pset = parcels.ParticleSet(
    fieldset, pclass=NestParticle, lon=X.flatten(), lat=Y.flatten()
)


def SampleNestID(particles, fieldset):
    particles.nestID = fieldset.NestID[particles]


pset.execute(SampleNestID, runtime=np.timedelta64(1, "s"), dt=np.timedelta64(1, "s"))

Indeed, the visualisation below shows that particles correctly identify the nest they are in based on their location.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

triang = mtri.Triangulation(
    uxda.uxgrid.node_lon.values,
    uxda.uxgrid.node_lat.values,
    triangles=uxda.uxgrid.face_node_connectivity.values,
)

plot_args = {
    "cmap": "viridis",
    "edgecolors": "k",
    "linewidth": 0.5,
    "vmin": 0,
    "vmax": 2.0,
}
ax.tripcolor(
    triang, facecolors=np.squeeze(uxda[0, :].values), shading="flat", **plot_args
)
ax.scatter(pset.lon, pset.lat, c=pset.nestID, **plot_args)
ax.set_aspect("equal")
ax.set_title("Nesting visualisation (triangulation and interpolated particle values)")
plt.tight_layout()
plt.show()

**Note for Joe:** Here it goes wrong. I think it's because of the UxGrid, wrapping around the antimeridian. See next cell.

In [ ]:
print(ds_tri.node_lon.values)
print(ux.Grid(ds_tri).node_lon.values)